In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import f_oneway
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

In [ ]:
! pip install xgboost

## Load the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
a1 = pd.read_excel('/content/drive/MyDrive/Python_Core_DA_ML_DS/Machine Learning/ML_Projects/Credit risk Modeling ML/case_study1.xlsx')
a2 = pd.read_excel('/content/drive/MyDrive/Python_Core_DA_ML_DS/Machine Learning/ML_Projects/Credit risk Modeling ML/case_study2.xlsx')


In [ ]:
df1 = a1.copy() # INternal data
df2 = a2.copy()# Cibil data

In [ ]:
df1.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32


In [ ]:
print(df1.shape)
print(df2.shape)

(51336, 26)
(51336, 62)


In [ ]:
df1.info(),df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

(None, None)

## Handling the null values (-99999)

### For df1 : Case_study 1

In [ ]:
# here we found no null values
df1.isnull().sum().sum(), df2.isnull().sum().sum()
# But in actually excel shet have null values with -99999 inform by data provider.

(0, 0)

In [ ]:
# found columns with -99999 values. (null)
list(df1.columns[df1.isin([-99999]).any()])

['Age_Oldest_TL', 'Age_Newest_TL']

In [ ]:
# check the no. of columns
print(df1[df1['Age_Oldest_TL'] == -99999].shape)
print(df1[df1['Age_Newest_TL'] == -99999].shape)
# very less values so we can remove it.

(40, 26)
(40, 26)


In [ ]:
df1 = df1[df1['Age_Oldest_TL'] != -99999 ]
print('51336 - 40 :',51336 - 40)
df1.shape

51336 - 40 : 51296


(51296, 26)

In [ ]:
# found columns with -99999 values. (null)
list(df1.columns[df1.isin([-99999]).any()])
# all values are clear...

[]

### For df1 case_study 2

In [ ]:
# found columns with -99999 values. (null)
df2_null_col = list(df2.columns[df2.isin([-99999]).any()])
df2_null_col
# all values are clear...

['time_since_recent_payment',
 'time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',
 'pct_currentBal_all_TL',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [ ]:
# here we found in excwl there is large number of null values with -99999
# we make solution:
# Aim  : at last we have just 80% data is enough for model
# 20 % of 50,000 row data
(50000 * 20)/100
# for null values > 10000 : we remove entire columns
# for null values <= 10000 : we remove rows from columns

10000.0

In [ ]:
# columns remove for  > 100000
df2_null_col = []
for i in df2.columns:
  if df2[df2[i] == -99999].shape[0] > 10000: # removce column if nul >100000
    df2_null_col.append(i)
print(len(df2_null_col))

df2 = df2.drop(columns=df2_null_col)

8


In [ ]:
# remove row for  <= 100000
for i in df2.columns:
  df2 = df2[df2[i] != -99999]

In [ ]:
# now check the shape of the data
df2.shape

(42066, 54)

## Merge both dataframe

In [ ]:
# Checking common column names
for i in df1.columns:
  for j in df2.columns:
    if i == j :
      print(i)

PROSPECTID


In [ ]:
# merge both dataframe with 'PROSPECTID'
df = pd.merge(df1,df2,how='inner',left_on='PROSPECTID',right_on='PROSPECTID')
df.shape

(42064, 79)

## Working on Categorical and Numerical columns

### Categorical columns

In [ ]:
# fetch only categorical columns
cat_col = df.select_dtypes(include=['object']).columns.tolist()
cat_col
# here our target columns is  : 'Approved_Flag'

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

In [ ]:
# Hypothesis Testing--------------------------------------------------
# find the relation of each columns with target columns
# step : 1--------
# H0 : null hypothesis : column is not associated with Approved_Flag
# H1 : alternate hypothesis : column is associated with Approved_Flag
# we have all categorical values [cat- cat] ---> use chi_square test --> find p values
# step : 2--------
# here in Bank domain alpha(strictness level) = 0.05
# High risk project : alpha --> low (0.001)
# step : 3--------
# confidence interval = 1 - alpha
# step : 4--------
# find value of p_cal
# Chi square : [cat - cat] ******************************** we select this
# T-test : [cat - num] : --> cat must have = 2
# ANOVA : [cat - num] : --> cat must have >= 3
# step : 5--------
# Goal : reject the hypothesis
# if p_val <= alpha : Reject the H0
# if p_val > alpha : Failed to reject the H0

In [ ]:
for i in ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']:
  print(i,'---',chi2_contingency(pd.crosstab(df[i],df['Approved_Flag'])).pvalue)
  # here all values are too small compare to alpha : 0.05 ---> we accept all columns
  # if we het more than > 0.05 , direct we can remove it from our dataframe.


MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


### Numerical Columns

In [ ]:
num_col = []
for i in df.columns:
  if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
    num_col.append(i)
len(num_col)

72

In [ ]:
# before going the comparision with target columns first compare each other.
# correlation
# we normally use : correlation --> but it is nature in linear relation ship
# it shows positive and negitive behaviour but not suitable convex function like y=m^2 --> parabolica curve
# it goes into misleading....

#_----------------------------------------------------------------------------------------------------------
# Multicolinearity
# in corporate used Multicolinearity: shows predactibility of each column by other column
# so which have high values then we remove it columns bcan no benefits to main same behaviour columns
# Goal : each colum is independent from each other
# we use VIF (variance inflation factor) ---> 1/(1-R^2) --> Range : (1 to inf)
# VIF : 0 --> No multicolinearity
# VIT : 1-5 --> Low multicolinearity
# VIT : 5-10  --> Moderate multicolinearity
# VIT : >10  --> High multicolinearity
# VIF 2 types : 1) Sequential(Right)  2) Parallel (Wrong)
#-------------------------------------------------------------------------------------------------------------
# In banking sector : threshold VIF = 6
# if VIF > 6 : remove columns
# if VIF <= 6 maintain columns

In [ ]:
vif_data = df[num_col]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0



for i in range (0,total_columns):

    vif_value = variance_inflation_factor(vif_data, column_index)
    # print (column_index,'---',vif_value)


    if vif_value <= 6:
        columns_to_be_kept.append( num_col[i] )
        column_index = column_index+1

    else:
        vif_data = vif_data.drop([ num_col[i] ] , axis=1)

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [ ]:
vif_data.shape
# from 72 ---> 39 means 33 columns are deducted

(42064, 39)

In [ ]:
df['Approved_Flag'].unique()

array(['P2', 'P1', 'P3', 'P4'], dtype=object)

In [ ]:
# Hypothesis --> Assocoated or not [ANOVA] --> [cat-num] --> cat have >=3
# Now perform the ANOVA for fro remaining 39 columns with traget columns
col_kept_num_anova = []

for i in columns_to_be_kept:
  a = list(df[i])
  b = list(df['Approved_Flag'])

  group1 = [value for value,group in zip(a,b) if group=='P1']
  group2 = [value for value,group in zip(a,b) if group=='P2']
  group3 = [value for value,group in zip(a,b) if group=='P3']
  group4 = [value for value,group in zip(a,b) if group=='P4']

  p_val_anova = f_oneway(group1,group2,group3,group4).pvalue

  if p_val_anova <= 0.05:
    col_kept_num_anova.append(i)


In [ ]:
len(col_kept_num_anova)

37

## Create the final dataframe

In [ ]:
final_features =  col_kept_num_anova + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'] + ['Approved_Flag']
df = df[final_features]
df.shape

(42064, 43)

## Encoding the Categorical Data

In [ ]:
for i in df.select_dtypes(include=['object']).columns.tolist()[:-1] :# no need of target columns
  print(i,'----',df[i].unique())

MARITALSTATUS ---- ['Married' 'Single']
EDUCATION ---- ['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
GENDER ---- ['M' 'F']
last_prod_enq2 ---- ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
first_prod_enq2 ---- ['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [ ]:
# Ordinal Encoding : Education
# SSC, OTHERs - 1
# 12TH - 2
# UNDER GRADUATE, GRADUATE, PROFESSIONAL  - 3
#  POST-GRADUATE - 4

# One hot encoding : MARITALSTATUS, GENDER, last_prod_enq2 ,first_prod_enq2

In [ ]:
df.loc[df['EDUCATION'] == 'SSC', ['EDUCATION']] = 1
df.loc[df['EDUCATION'] == 'OTHERS', ['EDUCATION']] = 1
df.loc[df['EDUCATION'] == '12TH', ['EDUCATION']] = 2
df.loc[df['EDUCATION'] == 'GRADUATE', ['EDUCATION']] = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE', ['EDUCATION']] = 3
df.loc[df['EDUCATION'] == 'PROFESSIONAL', ['EDUCATION']] = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE', ['EDUCATION']] = 4

In [ ]:
df['EDUCATION'] = df['EDUCATION'].astype(int)

In [ ]:
df['EDUCATION'].unique()

array([2, 3, 1, 4])

In [ ]:
# one hot encoding
df_encoded = pd.get_dummies(df,columns=['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'])

In [ ]:
df_encoded.shape

(42064, 55)

## Machine Learing model fitting

In [ ]:
# data preprocessing
X = df_encoded.drop(columns=['Approved_Flag'])
y = df_encoded['Approved_Flag']

In [ ]:
y.value_counts()
# Here target data is balance dataset so we can check Accuracy it is enough
# if Imbalance data : Check the F_1 score

Approved_Flag
P2    25452
P3     6440
P4     5264
P1     4908
Name: count, dtype: int64

In [ ]:
# Label Encoding of target column
le = LabelEncoder()
y_encoded =le.fit_transform(y)

In [ ]:
# split the data
X_train,X_test,y_train,y_test = train_test_split(X,y_encoded,test_size=0.2,random_state=42)


In [ ]:
# here fist we find base model with out any feature engineering
# after getting 1 best model we can implement standardscalling etc.....

### Decision Tree Classifier

In [ ]:
# 1 DecisionTreeClassifier
dt_model = DecisionTreeClassifier(max_depth=200, min_samples_split=10)
dt_model.fit(X_train,y_train)
y_pred_dt = dt_model.predict(X_test)

In [ ]:
# accuracy score,pricision,recall and f1_score
print('Accuracy : ', accuracy_score(y_test,y_pred_dt))
print()
precision,recall,f1_score,_ = precision_recall_fscore_support(y_test,y_pred_dt)
for i,j in enumerate(['P1','P2','P3','P4']):
  print(f"Class : {j}")
  print(f"Precision: {precision[i]}")
  print(f"Recall: {recall[i]}")
  print(f"F1 Score: {f1_score[i]}")
  print()
# NOt good for P3

Accuracy :  0.7090217520503982

Class : P1
Precision: 0.7214076246334311
Recall: 0.727810650887574
F1 Score: 0.7245949926362297

Class : P2
Precision: 0.810307662159514
Recall: 0.819623389494549
F1 Score: 0.8149389042175799

Class : P3
Precision: 0.3460639127045986
Recall: 0.3350943396226415
F1 Score: 0.3404907975460122

Class : P4
Precision: 0.6454183266932271
Recall: 0.6297376093294461
F1 Score: 0.6374815543531727



### Random Forest Classifier

In [ ]:
rf_model = RandomForestClassifier(n_estimators = 200, random_state=42)
rf_model.fit(X_train,y_train)
y_pred_rf = rf_model.predict(X_test)

In [ ]:
# accuracy score,pricision,recall and f1_score
print('Accuracy : ', accuracy_score(y_test,y_pred_rf))
print()
precision,recall,f1_score,_ = precision_recall_fscore_support(y_test,y_pred_rf)
for i,j in enumerate(['P1','P2','P3','P4']):
  print(f"Class : {j}")
  print(f"Precision: {precision[i]}")
  print(f"Recall: {recall[i]}")
  print(f"F1 Score: {f1_score[i]}")
  print()
# Not good for P3

Accuracy :  0.7636990372043266

Class : P1
Precision: 0.8370457209847597
Recall: 0.7041420118343196
F1 Score: 0.7648634172469203

Class : P2
Precision: 0.7957519116397621
Recall: 0.9282457879088206
F1 Score: 0.8569075937785909

Class : P3
Precision: 0.4423380726698262
Recall: 0.21132075471698114
F1 Score: 0.28600612870275793

Class : P4
Precision: 0.7178502879078695
Recall: 0.7269193391642371
F1 Score: 0.7223563495895703



### XGBClassifier

In [ ]:
xgb_model = XGBClassifier(objective='multi:softmax',num_class=4)
xgb_model.fit(X_train,y_train)
y_pred_xgb = xgb_model.predict(X_test)

In [ ]:
# objecive :-------------------------------
# "binary:logistic": The model learns to predict between two things, like 'Yes' or 'No'.
# "multi:softmax": The model learns to predict among multiple things, like different types of fruits.
# "multi:softprob": Similar to "multi:softmax", but instead of giving just one answer, it gives probabilities for each option.
#_----------------------------------------------------------------------------------------------------
# num_class :--------------------------------------
# num_class: This parameter is used specifically for multi-class classification problems. It indicates the number of classes in the dataset.
# For binary classification tasks, you don't need to specify this parameter, but for multi-class problems, you must set it to the number of classes.
# For example, if you have 3 classes in your dataset, you would set num_class=3

In [ ]:
# accuracy score,pricision,recall and f1_score
print('Accuracy : ', accuracy_score(y_test,y_pred_xgb))
print()
precision,recall,f1_score,_ = precision_recall_fscore_support(y_test,y_pred_xgb)
for i,j in enumerate(['P1','P2','P3','P4']):
  print(f"Class : {j}")
  print(f"Precision: {precision[i]}")
  print(f"Recall: {recall[i]}")
  print(f"F1 Score: {f1_score[i]}")
  print()
# Not good for P3

Accuracy :  0.7783192677998336

Class : P1
Precision: 0.823906083244397
Recall: 0.7613412228796844
F1 Score: 0.7913890312660173

Class : P2
Precision: 0.8255418233924413
Recall: 0.913577799801784
F1 Score: 0.8673315769665036

Class : P3
Precision: 0.4756380510440835
Recall: 0.30943396226415093
F1 Score: 0.3749428440786465

Class : P4
Precision: 0.7342386032977691
Recall: 0.7356656948493683
F1 Score: 0.7349514563106796



**ACCURACY ::::::::** <br>
1) DecisionTree Classifier : 71% <br>
2) RandomForestClassifer : 76% <br>
3) XGBClassifier : 78%<br>
--> So choose XGBClassifier model for Fine Tuning

## Hyperparameter Tuning

### XGBClassifier

In [ ]:
# Hyperparameter tuning in xgboost

In [ ]:
# Define the parameter grid for hyperparameter tuning

parameters = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

In [ ]:
xgb_model = XGBClassifier(objective='multi:softmax', num_class=4)

In [ ]:
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=parameters, cv=3, scoring='accuracy', n_jobs=-1)
# grid_search.fit(X_train, y_train)
# best : {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_class=4,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [ ]:
grid_search.best_params_

{'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}

In [ ]:
# with best parameters
new_xgb = grid_search.best_estimator_

In [ ]:
y_train_new = new_xgb.predict(X_train)
y_pred = new_xgb.predict(X_test)

In [ ]:
print("Train Accuracy : ",accuracy_score(y_train,y_train_new))
print("Test Accuracy : ",accuracy_score(y_test,y_pred))
# that is enough

Train Accuracy :  0.8062761879290363
Test Accuracy :  0.7811719957209081


In [1]:
import pandas as pd

In [2]:
data = pd.DataFrame(columns=['Name','Salary','City'])

In [10]:
Name=['Sugam','Kavya','Jatin']
Salary=[2000,25,8500]
City=['Pune','Mumbai','Patan']

In [11]:
extra_df = pd.DataFrame({'Name':Name,'Salary':Salary,'City':City})

In [13]:
extra_df
# now you can convert it into the the excel---> For checing the Trai and Test accuracy.....

,Name,Salary,City
0,Sugam,2000,Pune
1,Kavya,25,Mumbai
2,Jatin,8500,Patan
